# **ChatGLM 3**

# **Step 1: Libraries**

In [ ]:
%pip install datasets transformers --quiet

In [ ]:
from tqdm import tqdm
import pandas as pd
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments

# **Step 2: Load Data**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Medical_Transcription_Extraction/mtsamples.csv')

In [ ]:
transcription_list = df['transcription'].to_list()

# **Step 3: Model**

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm3-6b", trust_remote_code=True)
model = AutoModel.from_pretrained("THUDM/chatglm3-6b", trust_remote_code=True).half().cuda()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# **Step 4: Inference**

In [ ]:
def get_chatglm_response(transcription):
    prompt = f"""This is a patient's transcription: '{transcription}'
    For the given patient's transcription, do the next actions sequentially:
    First, propose the patient's age. If there is no words about age, return unknown.
    Second, propose no more than ten words that summarize the treatment. If you don't know the treatment, assume '''unknown'''
    Then, show your response following the next format:
    Age: <age>
    Treatment: <treatment>
    """
    # prompt = f"This is a patient's transcription: '{transcription}' \n\n If the treatment has already been administered, it's also considered as the treatment, the format will be revised slightly: The patient got [Your extracted answer] as treatment. If there is no specific neither age or treatment , you can return unknown, unknown. If there is no specific age but treatment, you can return unknown, The patient will get [Your extracted answer] as treatment. There may be not only one treatment. Please use this format as the answer: The patient age is [Your extracted answer]. The patient will get [Your extracted answer] as treatment."
    response, history = model.chat(tokenizer, prompt, history=[])
    return response

In [ ]:
model = model.eval()
res = []
for transcription in tqdm(transcription_list[0:2]):
    response = get_chatglm_response(transcription)
    res.append(response)
print(res)